- N-Gram Language Models: 2 Problems
    - Data Sparsity &  Long-term Dependency
    - 이 두 문제의 연관성과 동시에 해결하는게 쉽지 않았음을 알았다.
    - Neural N-Gram을 사용하면 2개를 동시에 해결하도록 노력할 수 있다.

### Neural N-Gram Language Model

<img  src="./image/img_NNgramLM.PNG" width="65%">

- Data Sparsity의 해결을 위한 모델 , 2000년대 초 쯤 제안된 모델 [Bengio et al., 2001]
- 지금까지 나온 figure와 거의 같다.
    - input이 n-gram이며, N개의 previous token이 있을 때, 새로 나와야하는 token의 prob을 계산하는 것이니까 $ \to $ supervised(text classification)와 같다 했었다.
- 그리고 Sentence Representation을 위해서는 단순하게 Concatenation을 한다.
    - 앞의 N개가 input으로 길이가 같다. 마지막 토큰은 타겟으로 사용하므로.

- 이 방법이 기존의 Count-based n-gram Language Model에 비해서, training set에서 나오지 않았던 n-gram에 대해서도 probability 계산이 가능하다.
    - 기존에서는 count가 0이여서 계산이 안됬었다. 왜 안됬던게 되냐?
    - Data Sparsity가 생긴 이유는 discrete space에서 counting 했기 때문이다. 토큰들이 여러개 있을 때 비슷한 토큰끼리든 아니든 모두 거리가 같았으니까! 이러면 아무리 N-gram이 많았어도 새로운 N-gram이 있을 때, 어떤 N-gram과 비슷한지 몰라 score를 줄 수 없었다.

- ex) Problem: c("Chasing a llama") = 0
    - Observation: c("Chasing a cat") $ \gg $ 0
    - Observation: c("Chasing a dog") $ \gg $ 0
    - Observation: c("Chasing a deer") $ \gg $ 0

- 이 Neural 모델은 어떻게 해결했냐. (간단함) 실제로 "Chasing a llama"는 구글북스에서도 counting 0이다.
    - 그 아래 고양이, 개, 사슴은 많이 나온다. 이 비슷한 것들을 알 수 있다면, 아마 아래 3개와 비슷하 prob를 갖지 않을까? 같은 guess를 한다.
    - How? NN은 Token Representation을 보고 Sentence Representation을 보는데,  Representation 자체가 Countinuous space에 있기 때문에 비슷한 애들은 가까이 붙는다.
    - Token Representation에서 포유류는 가까이 붙어있을 것이니까 어느 순간 확률을 계산하는 곳까지 쭉가면 비슷한 representaion을 가질 것이다.
    - 이러한 방법으로 "Countinuous Space"에서는 한번도 못본 n-gram token이어도  관측된 비슷한 토큰끼리 비슷한 distribution이 되도록 할 것이다.  
<img  src="./image/img_NNLM2.PNG" width="65%">

- Another Ex)
<img  src="./image/img_NNLM3.PNG" width="55%">

- 문장이 위처럼 딱 3개밖에 없을 때. 색칠된 단어에 초점을 맞춰 생각해보면
- 아래 Q의 질문에서 일반 count-based n-gram은 count가 0 이지만, Neural Language Model에서는 Countinuous Vector Space를 사용하면
    - 이 때, 어떤 input들이 어느 poitn에 들어가게 될지는 training하면서 구성이 될 것이다.
    - 먼저, Three가 Countinuous Vector Space의 어떤 point로 매핑이 되고, 그 매핑이 된 포인트에서부터 teams라는 단어의 prob이 무엇인지를 (높아지도록) 만들 것이다.
    - 그 다음 four가 들어가서 trainig할 때 이도 똑같이 teams라는 단어의 prob이 높아지도록 할 것이니까, three와 four를 하나의 포인트로 매핑하는 것이 optimal한 behavior일 것이다. 
    - 이것이 의미하는 것은 "three teams"와 "four teams"을 NN이 training함으로 써 NN의 token 또는 phrase Representation이 비슷한 phrase에서는 가까이 붙게 되고, 다른 phrase에서는 떨어지게 되는 것이다.
        - 여기서, 비슷함의 정의는 phrase이 후의 다음 단어를 predict하는데 비슷한게 나오느냐 안나오느냐로 similarity가 나오는 것이다.
    - 이 후 "four groups"가 왔을 때 three와 four가 비슷한 위치될 것이고, teams와 groups에 양쪽에 높은 prob을 주도록 될 것이다.
    - 이런 방식으로  Data Sparsity 이슈를 해결한다.


- In practice, (실전 사용(training) 방법)
    - Collect all n-grams from the corpus.(데이터 셋을 모운다.)
    - Shuffle all the n-grams to build a training set (모든 n-gram을 추출)
    - Train the neural n-gram language model using stochastic gradient descent on minibatches containing 100-1000 n-grams.
        - SGD를 위해 최대한 uniform하게 뽑아야하니까 shuffle하고 앞 순서부터 mini-batch
    - Early-stop based on the validation set. (얼스탑)
    - Report perplexity on the test. (Language Model의 performance measure는 Perplexity를 사용한다. 의미는 context를 봤을 때, 그 다음 단어를 얼마나 적은 subset안에서 고를 수 있는지. Perplexity=10이면 10개내외에서 다음 단어를 맞출 수 있다는 것. 낮을 수록 scoring을 잘한다고 생각 가능. 그러나 실제로는 조금 사용하긴 애매)
        - ppl $ = b^{{{1}\over{|D|}} \sum_{(x_{1},...,x_{D})\in{D}} log_{b} p(x_{N}|x_{1},...,x_{N-1})} $

- But 아직 n-gram을 사용하므로 Long-term에 대해 해결하진 못했다.
    - count-based와 다르게 N을 늘려 어느정도 해결할 수 있다. Data Sparsity issue가 생기지 않으니
    - But 늘리면, 모델 아키텍처 자체, 파라미터가 커진다.
    - 여기서 한계로 Increasing the context size – Convolutional Language Models [Kalchbrenner et al., 2015; Dauphin et al., 2016] CNN이 나온다!
    
        
### Convolutional Language Models - Increasing the context size

- convolution layer하나 올라갈 수록 하이어라키컬하게 보는게 확대가 된다.
- Dilated convolution to rapidly increase the window size.
    - 기존 CNN에서 예를들어 커널 3으로 하면은 너무 느리게 확장되어 모델의 깊이가 깊어 지므로 중간 꺼는 한칸 띠어서 보는 Idea이다.
<img  src="./image/img_DCLM1.PNG" width="55%">
    - 그러나 앞의 모델들과 달라 생기는 문제점: 앞의 모델은 지금까지 들어오는 token을 보고 다음 토큰을 predict하는 것인데 CNN이든 Dilated CNN을 쓰면 다음 토큰을 predict하는 데 이미 다음 토큰을 비롯해 미래의 토큰을 봐버린 것이므로
    - 이렇게 하면 exact probability decompose가 안되니까, Language모델이 아니다. 
    - 이걸 해결하기 위해서, 구글 딥마인드나 페북 에아리서치에서의 논문은 매번 predict할 때 마다 미래의 단어들에 대해서는 mask out을 한다. 이렇게 해서 predict할 땐 항상 그 전의 token을 사용해 Language모델이 도록!
        - [Kalchbrenner et al., 2015; Dauphin et al., 2016] 뒤에 것이 내가 했던 C-Gan VC의 GLU에 대한 paper!
    - 매번 Masking out하는 것이 문제가 되겠는데? 하지만 구현 자체는 masking pattern을 자동적으로 어렵지 않게 할 수 있다.
<img  src="./image/img_DCLM2.PNG" width="55%">

- 장점: Computation Efficiency, N은 늘리지만 파라미터 개수가 많지 않은 점
- 이아이디어를 기반으로 ByteNet, PixelCNN, WaveNet 등 이 나온다.
    - cf) 안드로이드 폰에는 이미 WaveNet이 들어가 있다.(구글 어시스턴트)
- 또 Gated Convlutional Language Model [Dauphin et al., 2016]
    - 계산할 때, Non-Linear Function을 좀더 Special하게 GLU를 썼던 것. (Cycle Gan VC에서 사용 됬던.) [CycleBEGAN VC](https://github.com/NamSahng/SingingStyleTransfer)
- Masking out의 의미: Causal Structure(인과 관계)를 Capture.
    - Sentence = sequence of Token 일 때, 앞에 나온 토큰으로 그 다음 토큰이 나오는 (미래의 토큰을 보고 과거를 generate 할 수 없으니)

- 앞에서 배운 Sentence Representation을 Language모델에 사용할 때는 과거, 미래 구분 해야하므로 사용해야할 때 굉장히 조심해야한다. 

- Causal sentence representation and language modelling
<img  src="./image/img_LMGeneration.PNG" width="65%">
    - Beginning of Sentence token은 주어진다 보고, 순차적으로 token을 뽑아 낸다.
    - 마지막 End of Sentence가 나오면, Sentence하나가 생성된 것이다.
    - 이러한 Causal structure만 따라가면, 다른 sentence representation(CNN, Neural N-Gram Language Model이 아닌) Language 모델을 계속 만들 수도 있고, context size자체도 infinite하게 늘릴 수 있겠다.
    
### CBoW Language Models - Infinite context $ n \to \infty $
    
- CBoW를 사용해서 모든 토큰을 순서를 무시하는 set으로 보고 averaging한다. 다만  Causal structure를 따라가도록.
- Text Classification에서는 굉장히 효과적이지만, Language Model에서는 효과적이지 않다.
    - Text Classification에서는 의미 중점적일 수 있지만, Language Model 의미 뿐만 아니라 구조자체가 중요한 문제이므로 순서를 무시하는 sentence representation은 적합하지 않으니까.
- CBoW는 Token Representation을 잘 찾는점에 word2vec과 같은 word embedding에만 많이 사용한다.

### Recurrent Language Models - Infinite context $ n \to \infty $

- Recurrent neural network based language model [Mikolov et al., 2010] 

- n-gram LM의 문제점은 Long-term depend.이니까, 이를 capture하기 위해 context를 infinite하게 늘려야하고 이를 위해 online으로 토큰을 하나하나 읽으면서 지금까지 읽은 모든 토큰을 하나의 vector representation으로 compress하는 방법이 필요하다는 방법을 보고 RNN을 쓰기로 하고 많은 사람들이 사용했다.
- sentence representation의 RNN과 다른 점: Bidirectional 하면 안된다는 것! 과거만 보고 summarize
- 현재는 가장 많이 쓰인다.
    - Speech Recognition, 핸드폰의 철자 오류, keyword suggestion 4 search Engine(근데 속도가 워낙 중요해 제한적으로 사용)
- 그리고 계속 scoring하므로 후보들을 prune out하는데도 많이 쓰인다.(자세하게 뒤에 나올 것이다)
    
### Recurrent Memory Networks - Infinite context $ n \to \infty $

- Recurrent Memory Networks for Language Modeling [Tran et al., 2016]
    - RNN + Self-Attention

- RNN이 잘쓰이지만, 문제점이 문장이 길어질 수록 RNN이 Prefix Summary하는 것이 점점 힘들어진다.
- RNN은 지금까지 본 sequence를 compress해 벡터 하나로 만드는 것이니, sequence가 길수록 기억해야할 내용이 많아지니까 RNN을 더 크게 만들어야하고, 파라미터 개수가 늘어나고, 데이터가 늘어나야하는 악순환이 이어진다.

- Self-Attention은  does not require such compression but still can capture long-term dependencies. 했다. $ \to $ RNN과 섞자.
    - RNN을 돌리고 그 위에서 Prob을 계산할 때는 Self-Attention을 써서 가장 중요한 애를 따로 뽑아 next token prob을 계산하는데 사용하자.

- 이렇게 함으로써 RNN이 지금까지 본 모든 토큰을 압축해야하는 이유가 없어졌다.
    - Sentence completion problem(문장에서 빈공간이 있을 때, 빈 공간을 맞추는 문제)에서는 거의 베스트오브 베스트라고 한다.
    
<img  src="./image/img_RNNSAT_LM.PNG" width="55%">

- 이런식으로 RNN, Self-Attention, CNN 이런 모든 것들이 DAG안에 들어만 가면 우리가 원하는데로 붙일 수 있어 섞어 쓰는 것이 가능하고, 문제에 따라 잘 섞으면 성능이 올라가는 효과가 있다.
    - RNMT+ 라는 최신 google의 Machine Translation 모델 또한 RNN과 Self-Attention을 합친 모델이었다. 

### In this lecture, we learned

- 우리가 집중한 부분 Language Model, sentence가 주어졌을 때 sentence의 score, prob. , 얼마나 likely한가를 보는 
- 여러가지 방법 중 autoregressive language model을 보았다.
    - 문장에서 각 토큰들의 prob을 쭉 online으로 읽으면서 계산하여 score를 계산하는 방법을 보았다.
    
- Autoregressive language model의 장점이 Language Model자체가 Unsupervised Learning인데, 이 문제를 Text Classification과 같은 Supervised Learning으로 바꾸었다는 것

- 그리고 Count-based에서 NN 모델들을 사용함으로서 문제들이 해결되는지를 보았다.

- Sentence representation을 먼저 배운 이유가 이를 활용해 Text Classification, Language Modeling, Machine Translation, Question & Answering 다양한 것들을 할 수 있다.

- 그리고 Sentence representation을 섞어서도 할 수 있음을 알았다!

### Questions 

- Q. 문제 setting을 Scoring과 Generation이 같다고 하면, 항상 같거나 비슷한 sentence만 말하는 것이 아니냐
- A. 지금까지 scoring을 deterministic한 generation을 말했는데, 이를 stochastic한 generation으로 바꿀 수 있다.
    - score를 모든 가능한 sentence들에다가 매긴 다음에 prob으로 만들어, 샘플링을 통하여 diversity를 줄 수 있다.
    
- Q. 문장 길이가 길어질 수록 점수가 낮아질 것 같은데 어떻게 해결하는지.
- A. MLE를 하다보면 Length distribution이 달라지므로, Estimation이 완벽하지 않다보니, 짧은 문장들에 대해 over estimate하고 긴문장들에 대해 under estimate하는 경향이 있는데 따라서 lenght penalty를 좀 주는데 이는 약간 task 에 따라 다르다.

- Q. Generic한 sentence representation은 어떻게 찾을 수 있냐.
- A. 이것이 sentence representation을 연구하는 사람들의 Holy Grail이다.
    - 학회에서의 결론은 sentence representation을 무얼 하고 싶냐에 따라 최적인 objective가 다 다르다.
    - GLUE라고 Evaluation 플랫폼을 만들어 놓아서, 새로운 sentence representation에대해 자동으로 수 많은 task들에대해 evaluation 해주는 것도 있다.
    - 궁극적으로 찾을 수 있을지 , Generic한 sentence representation이 있는지 아직 잘 모른다.

- Q. Pos-Tagging에서의 Self Attention 활용이 어떤지.
- A. Dictionary를 활용하는 방법으로 기릿해도 되지만, 딥러닝을 활용하자면 주어진 sentence를 summary한 후 다른 token과 비교하여 과연 이 토큰이 어떤 클래스 속해야하는지 봄으로 할 수 있다.
    - 다만 쓸 때 sentence당 classifer가 하나가 있는 것이 아니라 각 토큰마다 classifier가 있는 식으로 하면 되고, Self Attention으로 Tagging은 좋은 아이디어이다.

- Q. Self attention과 RNN의 계산량 이슈는 어떻게 해결하는지?
- A. Sentence와 Doc의 길이가 길 때, Test Time에는 RNN이 더 빠르다. 더 잘되느냐 는 다른 문제다.

- Q. 재밌는 문장은 어떻게 훈련해야할까? 
- A. Surprise를 먼저 define해야 한다. Language모델자체가 surprsie를 계산하는 것과 같다.
    - 이 ratio를 어떻게 맞춰야하나, scoring function을 잘 define해야겠지만
    - 하지만 syntax에서 surprise는 알아들을수 없으니 안되고, symantic에서 surprise하게 하여 잘 만들어 score를 잘 define하는 것이 중요할 것이다.



### Reference: 

- https://www.edwith.org/deepnlp 조경현교수님, 딥러닝을 이용한 자연어 처리 강의 및 강의 자료 

### Papers:

- "A Convolutional Neural Network for Modelling Sentences" [Nal Kalchbrenner et al., 2014]
- "Language Modeling with Gated Convolutional Networks" [Yann N. Dauphin et al., 2016]
- "Recurrent neural network based language model" [Mikolov et al., 2010]
- "Recurrent Memory Networks for Language Modeling" [Ke Tran et al., 2016] 

